In [1]:
!nvidia-smi

Wed Aug 17 15:40:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.95       Driver Version: 512.95       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   40C    P8    16W /  N/A |   1114MiB /  8192MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 计算设备

In [2]:
import torch
from torch import nn
torch.device('cpu'), torch.cuda.device('cuda'), torch.cuda.device('cuda:1')

(device(type='cpu'),
 <torch.cuda.device at 0x25bb5bac850>)

查询可用gpu的数量

In [3]:
torch.cuda.device_count()

1

这两个函数允许我们在不存在所需所有GPU的情况下运行代码。

In [5]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

# 张量与GPU
查询张量所在的设备

In [6]:
x = torch.tensor([1,2,3])
x.device

device(type='cpu')

# 存储在GPU上

In [12]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

假设你至少有两个GPU，下面的代码将在第二个GPU上创建一个随机张量。

In [11]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.8422, 0.2633, 0.6604],
        [0.6876, 0.4365, 0.0574]])

# 复制
如果我们要计算X + Y，我们需要决定在哪里执行这个操作。 例如，如 图5.6.1所示， 我们可以将X传输到第二个GPU并在那里执行操作。 不要简单地X加上Y，因为这会导致异常， 运行时引擎不知道该怎么做：它在同一设备上找不到数据会导致失败。 由于Y位于第二个GPU上，所以我们需要将X移到那里， 然后才能执行相加运算

由于此电脑只有一个gpu，就不搞花里胡哨了。。。

In [13]:
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [14]:
Y + Z  # 由于没有多个gpu，这几步就算了吧。。。

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [15]:
Z.cuda(0) is Z

True

# 神经网络与GPU

In [16]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

net(X)

tensor([[0.3231],
        [0.3231]], device='cuda:0', grad_fn=<AddmmBackward0>)

# 确认模型参数存储在同一个GPU上。

In [17]:
net[0].weight.data.device

device(type='cuda', index=0)